In [14]:
from gensim.models import KeyedVectors, Word2Vec

from preprocessing import *

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

Load vectors of word indices representing the question

In [ ]:
word2vec_embedding(token_train, token_test, 300)

In [6]:
w2v_train = pd.read_pickle("./data/word2vec_train.pkl")
w2v_test = pd.read_pickle("./data/word2vec_test.pkl")

In [7]:
w2v_train.head()

,id,qid1,tokenq1,qid2,tokenq2,is_duplicate
0,0,0,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]",241804,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10]",0
1,1,1,"[1, 2, 3, 12, 13, 13, 14]",13483,"[1, 15, 16, 17, 3, 18, 19, 20, 3, 13, 13, 14, 21]",0
2,2,2,"[22, 23, 24, 25, 3, 26, 27, 28, 29, 30, 31, 32]",241805,"[22, 23, 28, 26, 33, 34, 5, 35, 36, 37]",0
3,3,3,"[38, 39, 24, 40, 41, 42, 22, 23, 24, 43, 44]",241806,"[45, 3, 46, 47, 48, 49, 50, 49, 51, 48, 2, 52,...",0
4,4,4,"[53, 54, 55, 8, 56, 57, 58, 59, 60, 61, 62, 63]",241807,"[53, 64, 15, 65, 8, 59, 56]",0


Build the embedding matrix

In [10]:
# Load Google News vectors
googlenews_embedding = './data/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(googlenews_embedding, binary=True)

In [11]:
# Build the embedding matrix
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

vocabulary = dict()
inverse_vocabulary = ['<unk>']
stops = set(stopwords.words('english'))
questions = ['tokenq1', 'tokenq2']

# Iterate over the questions of both training and test datasets
for dataset in [token_train, token_test]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions:

            question_to_number = []  # q2n -> question numbers representation
            for word in row[question]:

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    question_to_number.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    question_to_number.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, question_to_number)

for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

NameError: name 'vocabulary' is not defined